In [1]:
import requests
from time import sleep
import pandas as pd


In [14]:
API_key = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
API_url = "https://api-datalab.coderslab.com/api/v2"
Auth = {'authorization': API_key}
SLEEP = 60/998

In [3]:
file_path = r"C:\Users\aga_c\Downloads\Pliki_do_Portfolio_Lab (1)\data\airports.csv"
aiports_file = pd.read_csv(file_path)

In [ ]:
file_path = r"C:\Users\aga_c\Downloads\Pliki_do_Portfolio_Lab (1)\data\airports.csv"
airports_df = pd.read_csv(file_path, delimiter=';')

if "origin_airport_id" not in airports_df.columns:
    raise ValueError("Plik airports.csv nie zawiera kolumny 'origin_airport_id'. Dostępne kolumny to: "
                     f"{', '.join(airports_df.columns)}")

print("Pierwsze wiersze danych:")
print(airports_df.head())

API_BASE_URL = "https://api-datalab.coderslab.com/api/v2"
API_KEY = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
HEADERS = {'Authorization': API_KEY}
SLEEP = 60/998

def fetch_airport_data(airport_id):
    url = f"{API_BASE_URL}/airport/{airport_id}"
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 404:
            print(f"Lotnisko o ID {airport_id} nie istnieje w API.")
            return None
        else:
            print(f"Błąd podczas pobierania danych dla lotniska {airport_id}: {response.status_code}")
            return None
    except requests.RequestException as e:
        print(f"Błąd sieci podczas pobierania danych dla lotniska {airport_id}: {e}")
        return None

airport_data = []

for airport_id in airports_df["origin_airport_id"].dropna().unique():
    try:
        airport_info = fetch_airport_data(airport_id)
        if airport_info:
            airport_data.append(airport_info)
    except Exception as e:
        print(f"Błąd podczas przetwarzania lotniska {airport_id}: {e}")

if airport_data:
    airport_df = pd.DataFrame.from_records(airport_data)

    
    output_file = "airport_data.csv"
    airport_df.to_csv(output_file, index=False)
    print(f"Dane lotnisk zapisane do pliku {output_file}.")
else:
    print("Brak danych do zapisania.")


In [ ]:
from dateutil.relativedelta import relativedelta

API_BASE_URL = "https://api-datalab.coderslab.com/api/v2"
API_KEY = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
HEADERS = {'Authorization': API_KEY}
SLEEP = 60/998


def get_weather_data_for_month(year, month):
    start_date = datetime(year, month, 1)
    next_month = start_date + relativedelta(months=1)
    end_date = next_month - relativedelta(days=1)

    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    
    url = f"{API_BASE_URL}/airportWeather"
    params = {
        "startDate": start_date_str,
        "endDate": end_date_str
    }
    response = requests.get(url, headers=HEADERS, params=params)
    
    if response.status_code == 200:
        return response.json()['data']  
    else:
        print(f"Error fetching data for {start_date_str} to {end_date_str}")
        return []

start_year = 2019
start_month = 1
end_year = 2020
end_month = 3

weather_data = []

current_year = start_year
current_month = start_month

while current_year < end_year or (current_year == end_year and current_month <= end_month):
    monthly_data = get_weather_data_for_month(current_year, current_month)
    weather_data.extend(monthly_data)  

    current_month += 1
    if current_month > 12:
        current_month = 1
        current_year += 1

weather_df = pd.DataFrame(weather_data)

weather_df.to_csv('airport_weather.csv', index=False)

print("Dane zostały zapisane do pliku 'airport_weather.csv'.")


In [ ]:
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

file_path = r"C:\Users\aga_c\Downloads\Pliki_do_Portfolio_Lab (1)\data\airports.csv"
API_BASE_URL = "https://api-datalab.coderslab.com/api/v2/flight"
API_KEY = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
HEADERS = {'Authorization': API_KEY}
SLEEP = 60 / 998  
START_DATE = "2019-01-01"
END_DATE = "2020-03-31"
MAX_WORKERS = 4  

airports_df = pd.read_csv(file_path, delimiter=';')
origin_airport_ids = airports_df['origin_airport_id'].unique()


date_range = pd.date_range(start=START_DATE, end=END_DATE, freq='MS').strftime("%Y-%m").tolist()


def fetch_flight_data(airport_id, date):
    params = {'airportId': airport_id, 'date': date}
    response = requests.get(API_BASE_URL, headers=HEADERS, params=params)
    time.sleep(SLEEP)

    if response.status_code == 200:
        data = response.json()
        for record in data:
            record['airport_id'] = airport_id
            record['month'] = date
        return data
    elif response.status_code == 204:
        print(f"Brak danych dla lotniska {airport_id} w miesiącu {date}.")
        return []
    else:
        print(f"Błąd {response.status_code} dla lotniska {airport_id} w miesiącu {date}.")
        return []

flight_data = []

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    future_to_request = {
        executor.submit(fetch_flight_data, airport_id, date): (airport_id, date)
        for airport_id in origin_airport_ids
        for date in date_range
    }

    for future in as_completed(future_to_request):
        try:
            data = future.result()
            if data:
                flight_data.extend(data)
        except Exception as e:
            airport_id, date = future_to_request[future]
            print(f"Błąd podczas przetwarzania lotniska {airport_id} w miesiącu {date}: {e}")

flight_df = pd.DataFrame(flight_data)

flight_df.to_csv("flight2.csv", index=False)
print("Dane zostały zapisane do pliku flight.csv.")
